## Check GPU

In [17]:
!nvidia-smi

Mon Mar 18 14:59:46 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 551.76                 Driver Version: 551.76         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1650      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8              8W /   50W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Install ultralytics

In [18]:
!pip install ultralytics

## Going Through Project

### Initilising Model

In [19]:
# import ultralytics
from ultralytics import YOLO

# Configure the tracking parameters and run the tracker
object_detector = YOLO('yolov8n.pt')  # Initializing YOLOv8 model with pre-trained weights ('yolov8n.pt')

### Opening Video File

In [20]:
import cv2  # Importing OpenCV for image processing tasks

input_video_path = "Videos/sample_video2.MOV"  # Path to the input video file
video_capture = cv2.VideoCapture(input_video_path)  # Opening the video file for reading

Get frames per second (fps) of the input video

In [21]:
fps = video_capture.get(cv2.CAP_PROP_FPS)  # Retrieve frames per second (fps) of the input video

Get dimensions of the input video

In [22]:
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))  # Width of the input video frame
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))  # Height of the input video frame

Define Output video file path

In [23]:
output_video_path = "Videos/Output/output.mp4"  # Path to the output video file

Define codec and create VideoWriter object

In [24]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Define the codec for the output video
output_video = cv2.VideoWriter(output_video_path, fourcc, fps, (video_width, video_height)) # Create a VideoWriter object to write the annotated frames to the output video

Define Coordinates of Region of Interest (ROI)

In [25]:
import numpy as np  # Importing NumPy for numerical computations

region_of_interest_Coordinates = np.array([[22, 154], [1602, 170], [1630, 1066], [42, 1054]], dtype=np.int32) # Define the coordinates of the Region of Interest (ROI) rectangles where people will be detected and tracked

Dictionary to store entry timestamps of people in the queue

In [26]:
people_entry_timestamps = {}  # Dictionary to store entry timestamps of people in the queue

List to store time spent by each person in the queue

In [27]:
time_spent_in_queue = []  # List to store time spent by each person in the queue

CSV file to store time spent by each person in the queue

In [28]:
import csv  # Importing CSV module for reading and writing CSV files

csv_filename = "queue_time.csv"  # Name of the CSV file to store time spent by each person in the queue
csv_file = open(csv_filename, 'w', newline='')  # Open the CSV file for writing
csv_writer = csv.writer(csv_file)  # Create a CSV writer object to write data to the CSV file

Initialize frame count

In [29]:
frame_count = 0  # Initialize frame count to keep track of the number of frames processed

### Loop through each frame in the video

In [30]:
while video_capture.isOpened():  # Iterate until the video ends or there are no more frames

    # Read a frame from the video
    success, frame = video_capture.read()  # Read a frame from the input video

    if success:  # Check if the frame was read successfully

        # Perform object detection and tracking using YOLOv8
        detection_results = object_detector.track(frame, persist=True) # Detect and track objects (people) in the frame using YOLOv8 model with persistence enabled

        # Extract bounding boxes and track IDs
        detected_boxes = detection_results[0].boxes.xyxy.cpu()  # Extract detected bounding boxes from detection results
        track_ids = detection_results[0].boxes.id.int().cpu().tolist() # Extract track IDs associated with the detected bounding boxes

        # Check if the center of bounding boxes is inside the region of interest (ROI)
        for box, track_id in zip(detected_boxes, track_ids):
            x1, y1, x2, y2 = box  # Extract bounding box coordinates (top-left and bottom-right corners)
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convert coordinates to integers

            x_center = (x1 + x2) / 2  # Calculate x-coordinate of the center of the bounding box
            y_center = (y1 + y2) / 2  # Calculate y-coordinate of the center of the bounding box

            # Check if the center of the bounding box is inside the ROI
            if cv2.pointPolygonTest(region_of_interest_Coordinates, (x_center, y_center), False) > 0:
                if str(track_id) not in people_entry_timestamps:
                    # Record entry timestamp of the person in the queue
                    people_entry_timestamps[str(track_id)] = str(frame_count)

                # Draw rectangle around the detected person
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, "Person ID: " + str(track_id), (x1, y1 - 5),
                            cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (20, 255, 0), 2)
                # Annotate the frame with bounding box and person ID

            else:  # If the person exits the ROI
                if str(track_id) in people_entry_timestamps:
                    # Record exit timestamp of the person from the queue
                    exit_timestamp = frame_count
                    entry_timestamp = people_entry_timestamps[str(track_id)]

                    # Calculate time spent by the person in the queue
                    time_spent = (exit_timestamp - int(entry_timestamp)) / fps
                    time_spent_in_queue.append(time_spent)

                    # Write time spent by the person to the CSV file
                    csv_writer.writerow(
                        ["Time spent by person " + str(track_id) + " in line is " + str(time_spent)])

                    # Remove entry from the dictionary
                    people_entry_timestamps.pop(str(track_id))

        # Draw ROI rectangles on the frame
        cv2.drawContours(frame, [region_of_interest_Coordinates], -1, (255, 0, 0), 3)
        # Draw the region of interest (ROI) rectangles on the frame for visualization

        # Write annotated frame to the output video
        output_video.write(frame)  # Write the annotated frame to the output video

        # Increment frame count
        frame_count += 1  # Increment the frame count for the next frame

    else:  # If there are no more frames in the video
        break  # Break the loop

0: 384x640 1 person, 59.5ms
Speed: 3.0ms preprocess, 59.5ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 54.0ms
Speed: 3.0ms preprocess, 54.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 57.2ms
Speed: 3.1ms preprocess, 57.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 63.6ms
Speed: 2.0ms preprocess, 63.6ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 61.1ms
Speed: 3.0ms preprocess, 61.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 55.5ms
Speed: 2.4ms preprocess, 55.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 54.9ms
Speed: 2.0ms preprocess, 54.9ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)
0: 384x640 1 person, 56.5ms
Speed: 3.0ms preprocess, 56.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)


Release video capture, output video, and CSV file resources

In [31]:
video_capture.release()  # Release the video capture object
output_video.release()  # Release the output video object
csv_file.close()  # Close the CSV file

Print the path to the output video file

In [32]:
print(f"Output video saved at: {output_video_path}")  # Print the path to the output video file

Output video saved at: Videos/Output/output.mp4
